In [1]:
import os
import pandas as pd
import numpy as np
from math import sqrt
from tqdm import tqdm

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

### Load Dataset

In [2]:
path = '../data'
ratings_df = pd.read_csv(os.path.join(path, 'ratings.csv'), encoding='utf-8')

print(ratings_df.shape)
ratings_df.head(2)

(100836, 4)


,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247


In [3]:
train_df, test_df = train_test_split(ratings_df,
                                     test_size=0.2,
                                     random_state=1234)
print(train_df.shape)
print(test_df.shape)

(80668, 4)
(20168, 4)


### Sparse Matrix 만들기

In [5]:
sparse_matrix = train_df.groupby('movieId').apply(lambda x : pd.Series(x['rating'].values, index=x['userId']))
sparse_matrix = sparse_matrix.unstack()

print(sparse_matrix.shape)
sparse_matrix.head(2)

(8938, 610)


userId,1,2,3,4,5,6,7,8,9,10,...,601,602,603,604,605,606,607,608,609,610
movieId,,,,,,,,,,,,,,,,,,,,,
1,NaN,NaN,NaN,NaN,4.0,NaN,4.5,NaN,NaN,NaN,...,4.0,NaN,4.0,3.0,4.0,2.5,NaN,2.5,NaN,5.0
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,NaN,NaN,...,NaN,4.0,NaN,NaN,3.5,NaN,NaN,2.0,NaN,NaN


### Cosine Similarity

In [6]:
from sklearn.metrics.pairwise import cosine_similarity

def cos_sim_matrix(a, b):
    cos_sim_values = cosine_similarity(a.values, b.values)
    
    return pd.DataFrame(data=cos_sim_values,
                         columns=a.index,
                         index=a.index)

### Neighborhood-based Collaborative Filtering

### 1) Item-based

In [7]:
item_sparse_matrix = sparse_matrix.fillna(0)

item_sparse_matrix.shape

(8938, 610)

In [8]:
item_cossim_df = cos_sim_matrix(item_sparse_matrix, item_sparse_matrix)

print(item_cossim_df.shape)
item_cossim_df.head(2)

(8938, 8938)


movieId,1,2,3,4,5,6,7,8,9,10,...,190219,191005,193565,193567,193571,193573,193579,193581,193587,193609
movieId,,,,,,,,,,,,,,,,,,,,,
1,1.000000,0.304336,0.267816,0.040259,0.221228,0.266544,0.149392,0.132943,0.182044,0.296838,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.304336,1.000000,0.226138,0.052482,0.154783,0.209716,0.189420,0.068012,0.027945,0.303157,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [9]:
# movie id : 8938
# user id : 610
item_prediction_result_df = pd.DataFrame(index=item_sparse_matrix.columns,
                                         columns=item_sparse_matrix.index)

item_prediction_result_df

movieId,1,2,3,4,5,6,7,8,9,10,...,190219,191005,193565,193567,193571,193573,193579,193581,193587,193609
userId,,,,,,,,,,,,,,,,,,,,,
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
606,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
607,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
608,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [10]:
def func(data):
    # (1 x n) x (n x 8938) => (1 x 8938)
    # (유저가 본 영화 평점 * 유저가 본 영화와의 유사도) / 유사도 총 합
    user_data = train_df[train_df['userId'] == data.name]
    user_sim = item_cossim_df.loc[user_data['movieId'].values].values
    
    item_prediction_result_df.loc[data.name] = np.matmul(user_data['rating'].values.reshape(1,-1), user_sim).squeeze() / (user_sim.sum(0)+1)
    
item_prediction_result_df.apply(func, axis=1)

item_prediction_result_df.head(10)

movieId,1,2,3,4,5,6,7,8,9,10,...,190219,191005,193565,193567,193571,193573,193579,193581,193587,193609
userId,,,,,,,,,,,,,,,,,,,,,
1,4.20279,4.18783,4.17609,3.03272,4.04561,4.24019,3.9933,3.90547,3.70279,4.18855,...,0.979003,0,0,0,0,0,0,0,0,1.38273
2,3.19261,3.15207,2.46138,0.280903,2.61916,2.9754,1.85877,1.68768,1.09268,3.02235,...,0.387743,1.70885,1.70885,1.70885,1.70885,1.70885,1.70885,1.70885,1.70885,2.32666
3,1.32315,1.20446,1.39635,0.217016,0.819716,1.59889,0.802514,0.755008,0.86708,1.55365,...,0,0,0,0,0,0,0,0,0,0
4,3.48994,3.48805,3.38864,3.15017,3.30488,3.50267,3.31794,2.90629,3.03413,3.41697,...,1.0335,0.505535,0.505535,0.505535,0.505535,0.505535,0.505535,0.505535,0.505535,1.963
5,3.27916,3.1278,2.99947,2.56582,2.94179,3.16869,3.00793,2.67809,2.27668,3.01557,...,0.391638,0,0,0,0,0,0,0,0,0.562705
6,3.58597,3.57233,3.53615,3.33866,3.51444,3.5477,3.53826,3.42057,3.3406,3.55275,...,0.446497,0,0,0,0,0,0,0,0,0.672442
7,3.36009,3.25338,3.24516,2.22581,3.2114,3.33808,3.1832,2.8918,2.89194,3.3515,...,0.872628,0.3554,0.3554,0.3554,0.3554,0.3554,0.3554,0.3554,0.3554,2.50752
8,3.26123,3.22385,3.03702,2.58129,2.94548,3.21505,3.06126,2.6713,2.37,3.17426,...,0.597653,0,0,0,0,0,0,0,0,0.730709
9,2.80797,2.62156,2.28209,0.767416,2.25043,2.64179,2.04582,1.59945,1.07072,2.63652,...,0.458811,0,0,0,0,0,0,0,0,0


### 2) User-based

In [11]:
user_sparse_matrix = sparse_matrix.fillna(0).T

user_sparse_matrix.shape

(610, 8938)

In [12]:
user_cossim_df = cos_sim_matrix(user_sparse_matrix, user_sparse_matrix)

print(user_cossim_df.shape)
user_cossim_df.head(2)

(610, 610)


userId,1,2,3,4,5,6,7,8,9,10,...,601,602,603,604,605,606,607,608,609,610
userId,,,,,,,,,,,,,,,,,,,,,
1,1.000000,0.016665,0.07057,0.160438,0.07541,0.087404,0.123664,0.069225,0.037416,0.009900,...,0.056163,0.124896,0.162324,0.050852,0.101583,0.128198,0.240652,0.225897,0.063984,0.095228
2,0.016665,1.000000,0.00000,0.004295,0.02056,0.030996,0.027726,0.000000,0.000000,0.057112,...,0.151666,0.019379,0.006645,0.000000,0.000000,0.028180,0.000000,0.046286,0.033522,0.090288


In [13]:
user_prediction_result_df = pd.DataFrame(index=user_sparse_matrix.columns,
                                         columns=user_sparse_matrix.index)
user_prediction_result_df.shape

(8938, 610)

In [14]:
def func(data):
    
    movie_data = train_df[train_df['movieId'] == data.name]
    movie_sim = user_cossim_df.loc[movie_data['userId'].values]
    
    user_prediction_result_df.loc[data.name] = np.matmul(movie_data.rating.values,
                                                         movie_sim.values) / (movie_sim.sum(0)+1)

user_prediction_result_df.apply(func, axis=1)
user_prediction_result_df = user_prediction_result_df.T

user_prediction_result_df.head(10)

movieId,1,2,3,4,5,6,7,8,9,10,...,190219,191005,193565,193567,193571,193573,193579,193581,193587,193609
userId,,,,,,,,,,,,,,,,,,,,,
1,3.67043,3.01898,3.02703,0.636151,2.33427,3.65344,2.54601,1.01293,1.59897,3.2986,...,0.0654522,0,0,0,0,0,0,0,0,0.140421
2,3.34895,2.57281,1.7197,0.146062,1.55944,2.9715,1.49602,0.289957,0.408521,2.88504,...,0.0463527,0.427493,0.332495,0.284995,0.379994,0.379994,0.332495,0.379994,0.332495,0.592074
3,2.121,1.27956,1.26626,0.0367179,0.466202,2.1174,0.780114,0.104695,0.218952,1.8424,...,0,0,0,0,0,0,0,0,0,0
4,3.61963,2.93753,2.73966,0.72914,2.1024,3.50549,2.50204,0.613417,1.2701,3.09378,...,0.0574532,0.052407,0.040761,0.034938,0.046584,0.046584,0.040761,0.046584,0.040761,0.251308
5,3.76122,3.21044,2.92431,0.957915,2.54943,3.45657,2.85021,1.20054,1.5878,3.13192,...,0.0586848,0,0,0,0,0,0,0,0,0.142512
6,3.83087,3.32198,3.32667,1.21591,3.03136,3.60413,3.11031,1.63385,2.15392,3.25954,...,0.0290969,0,0,0,0,0,0,0,0,0.0660016
7,3.77535,3.10961,2.83467,0.572114,2.35059,3.64751,2.62267,0.940899,1.55949,3.36406,...,0.0674776,0.0409324,0.0318363,0.0272883,0.0363844,0.0363844,0.0318363,0.0363844,0.0318363,0.58529
8,3.79756,3.3381,3.12298,1.16575,2.71027,3.53906,3.00015,1.44235,1.66749,3.22467,...,0.0958472,0,0,0,0,0,0,0,0,0.202119
9,3.17211,2.35781,1.68523,0.0990461,1.19739,2.80897,1.40665,0.275126,0.327804,2.82822,...,0.0636902,0,0,0,0,0,0,0,0,0


In [15]:
item_prediction_result_df.shape, user_prediction_result_df.shape

((610, 8938), (610, 8938))

### Evaluation (RMSE)

In [16]:
test_df.head(2)

,userId,movieId,rating,timestamp
99731,610,3527,5.0,1479545223
97583,606,1250,3.5,1171376891


In [18]:
len(set(test_df.userId.unique()) - set(train_df.userId.unique()))

0

In [19]:
len(set(test_df.movieId.unique()) - set(train_df.movieId.unique()))

786

In [27]:
# 학습에 사용된 movieId에 한해서만 평가 진행
check_lst = set(test_df.movieId.unique()) & set(train_df.movieId.unique())
test_df = test_df[test_df.apply(lambda x : x.movieId in check_lst, axis=1)]

test_df.shape

(19316, 4)

In [36]:
test_df['item_prediction_rmse'] = test_df.apply(lambda x: item_prediction_result_df[x.movieId][x.userId], axis=1)
test_df['user_prediction_rmse'] = test_df.apply(lambda x: user_prediction_result_df[x.movieId][x.userId], axis=1)

test_df.head(3)

,userId,movieId,rating,timestamp,item_prediction_rmse,user_prediction_rmse
99731,610,3527,5.0,1479545223,3.797414,3.285668
97583,606,1250,3.5,1171376891,3.669890,3.527574
38197,262,213,5.0,840310907,1.793772,0.994251


In [41]:
# RMSE 계산
N = test_df.shape[0]

item_prediction_rmse = (test_df.apply(lambda x : (x.rating-x.item_prediction_rmse)**2, axis=1).sum() / N) ** 0.5
user_prediction_rmse = (test_df.apply(lambda x : (x.rating-x.user_prediction_rmse)**2, axis=1).sum() / N) ** 0.5

print('item rmse : {:.4f}'.format(item_prediction_rmse))
print('user rmse : {:.4f}'.format(user_prediction_rmse))

item rmse : 0.9389
user rmse : 1.4137
